# Testing integration with `vegas`

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ October 11, 2022

Trying to do a spectroscopic overlap calculation evaluating the integrals in the $\delta U^\dagger$ term with Monte Carlo integration using `vegas`. Currently attempting to do all the sums in the expression by looping (excluding as many terms equal to zero as possible), and use MC only for integration over momenta.

_Last update:_ October 25, 2022

In [1]:
import numpy as np
import numpy.linalg as la
from sympy.physics.quantum.cg import CG
import vegas

Testing `vegas` with example from tutorial [online](https://vegas.readthedocs.io/en/latest/tutorial.html)

In [2]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [3]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.07(70)        1.07(70)            0.00     1.00
  2   1.28(60)        1.19(46)            0.05     0.82
  3   1.15(11)        1.15(10)            0.03     0.97
  4   0.892(51)       0.943(46)           1.69     0.17
  5   0.960(36)       0.954(28)           1.29     0.27
  6   1.060(24)       1.015(18)           2.66     0.02
  7   1.001(18)       1.008(13)           2.26     0.03
  8   1.027(16)       1.016(10)           2.06     0.04
  9   1.011(12)       1.0140(78)          1.82     0.07
 10   1.006(12)       1.0116(65)          1.65     0.10

result = 1.0116(65)    Q = 0.10


Testing integral from degenerate electron gas in Fetter and Walecka section 1.3

In [20]:
def heaviside(x):
    if x >= 0:
        return 1
    else:
        return 0

In [21]:
def g(Q_vector):
    kF = 1
    q_vector = np.repeat(kF/np.sqrt(3),3)
#     integrand = (heaviside(kF-la.norm(Q_vector+q_vector/2))
#                  * heaviside(kF-la.norm(Q_vector-q_vector/2))
#                  * float(CG(0,0,1/2,1/2,1/2,1/2).doit()))
    integrand = (heaviside(kF-la.norm(Q_vector+q_vector/2))
                 * heaviside(kF-la.norm(Q_vector-q_vector/2))
                 * 1)
                    
    return integrand

In [22]:
integ = vegas.Integrator([[-3,3],[-3,3],[-3,3]])

result = integ(g, nitn=20, neval=10000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))
print(f'True result = {5*np.pi/12}')

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.32(11)        1.32(11)            0.00     1.00
  2   1.09(12)        1.210(83)           1.82     0.18
  3   1.201(82)       1.205(59)           0.92     0.40
  4   1.240(43)       1.228(35)           0.69     0.56
  5   1.370(72)       1.255(31)           1.31     0.26
  6   1.329(26)       1.299(20)           1.70     0.13
  7   1.242(19)       1.269(14)           2.12     0.05
  8   1.316(17)       1.288(11)           2.45     0.02
  9   1.55(28)        1.288(11)           2.26     0.02
 10   1.52(22)        1.289(11)           2.14     0.02
 11   1.274(12)       1.2822(82)          2.00     0.03
 12   1.311(14)       1.2896(71)          2.11     0.02
 13   1.2825(93)      1.2870(56)          1.97     0.02
 14   1.303(11)       1.2903(50)          1.94     0.02
 15   1.2916(78)      1.2907(42)          1.80     0.03
 16   1.48(19)        1.2908(42)          1.75  

In [23]:
print(result)
print(result.mean)
print(result.sdev)

1.2873(30)
1.2872676547874853
0.002954842881264417


In [8]:
# i = 0
# j_array = np.arange(0, 3, 1/2)
# print(j_array)
# for j1 in j_array:
#     for mj1 in np.arange(-j1, j1+1, 1):
#         for j2 in j_array:
#             for mj2 in np.arange(-j2, j2+1, 1):
#                 for j3 in np.arange(abs(j1-j2), j1+j2+1, 1):
#                     mj3 = mj1+mj2
#                     cg = float(CG(j1, mj1, j2, mj2, j3, mj3).doit())
#                     print(f"j1={j1}, mj1={mj1}, j2={j2}, mj2={mj2}, j3={j3},"
#                           f" mj3={mj3}, CG = {cg}")
#                     i +=1
# print(i)

In [9]:
# Testing SymPy Clebsch-Gordan function
float(CG(0.5, 0.5, 0.5, -0.5, 1, 0).doit())

0.7071067811865476

Testing evaluation of sums over quantum numbers with many Clebsch-Gordan coefficients

In [10]:
def get_channel_quantum_numbers(channel):
    
    S = int((int(channel[0])-1)/2)
    J = int(channel[2])
    
    if channel[1] == 'S':
        Lp = 0
    elif channel[1] == 'P':
        Lp = 1
    else:
        Lp = 2
    
    if len(channel) > 3:
        
        if channel[5] == 'S':
            L = 0
        elif channel[5] == 'P':
            L = 1
        else:
            L = 2
            
    else:
        
        L = Lp
            
    return S, Lp, L, J

In [11]:
def get_total_isospin(S, L):
    
    if (1-(-1)**(L+S)) == 0:
        T = 1
    else:
        T = 0
        
    return T

In [12]:
def get_sp_quantum_numbers(sp_state):
    
    n = int(sp_state[0])
    
    if sp_state[1] == 's':
        l = 0
    elif sp_state[1] == 'p':
        l = 1
    elif sp_state[1] == 'd':
        l = 2
    else:
        l = 3
        
    j_str = sp_state[2:]
    if j_str == '0p5':
        j = 1/2
    elif j_str == '1p5':
        j = 3/2
    else:
        j = 5/2
        
    return n, l, j

In [13]:
def print_quantum_numbers(
        channel, J, M_J, Lp, M_Lp, L, M_L, S, M_S, T, M_T, tau_alpha, tau_beta,
        tau_gamma, sigma_alpha, sigma_beta, sigma_gamma, sigma_betap,
        sigma_gammap, n_gamma, j_gamma, m_j_gamma, l_gamma, m_l_gamma,
        m_l_gammap, n_beta, j_beta, m_j_beta, l_beta, m_l_beta,
        m_l_betap):
    
    print(channel)
    print(f" J={J}, M_J={M_J}, L'={Lp}, M_L'={M_Lp}, L={L}, M_L={M_L},"
          f" S={S}, M_S={M_S}, T={T}, M_T={M_T}")
    print(f" tau_alpha={tau_alpha}, tau_beta={tau_beta},"
          f" tau_gamma={tau_gamma}")
    print(f"  sigma_alpha={sigma_alpha}, sigma_beta={sigma_beta},"
          f" sigma_gamma={sigma_gamma}, sigma_beta'={sigma_betap},"
          f" sigma_gamma'={sigma_gammap}")
    print(f"   n_gamma={n_gamma}, j_gamma={j_gamma}, m_j_gamma={m_j_gamma},"
          f" l_gamma={l_gamma}, m_l_gamma={m_l_gamma}, m_l_gamma'={m_l_gammap}")
    print(f"    n_beta={n_beta}, j_beta={j_beta}, m_j_beta={m_j_beta},"
          f" l_beta={l_beta}, m_l_beta={m_l_beta}, m_l_beta'={m_l_betap}")

In [14]:
# # channels = (
# #     '1S0', '3S1-3S1', '3S1-3D1', '3D1-3S1', '3D1-3D1', '3P0', '3P1', '1P1'
# # )
# channels = ('1S0', '3S1-3S1')

# spins = (1/2, -1/2)
# isospins = (1/2, -1/2)

# total_spins = (1, 0, -1)

# alpha = '1s0p5'
# tau_alpha = 1/2
# # betas = ['1s0p5', '1p1p5', '1p0p5', '2s0p5', '2p1p5', '2p0p5']
# betas = ['1s0p5']
# gammas = ['1s0p5']

# n_alpha, l_alpha, j_alpha = get_sp_quantum_numbers(alpha)
# m_j_alpha = j_alpha

In [15]:
# # First attempt: there are still many contributions equal to zero here

# i = 0

# tau_beta = tau_alpha

# # Compute k4_vector, k4, \theta_4, \phi_4
# # Compute kp_vector, kp, \theta', \phi'
# # Compute k_vector, k, \theta, \phi

# # Compute \phi_\alpha(k3)

# # DON'T FORGET FACTOR OF 1/np.sqrt(2)

# for channel in channels:
    
#     S, Lp, L, J = get_channel_quantum_numbers(channel)
#     T = get_total_isospin(S, Lp)
    
#     # Compute 1-(-1)^(L'+S+T) and 1-(-1)^(L+S+T)
#     factor = (1-(-1)**(Lp+S+T)) * (1-(-1)**(L+S+T))
    
#     # Compute \delta U^\dagger(k', k)
    
#     M_Js = np.arange(-J, J+1, 1)
#     for M_J in M_Js:
    
#         if T == 0:
            
#             tau_gamma = -tau_alpha
#             M_T = 0
            
#             # Compute < \tau_\beta \tau_\gamma | T M_T >
#             isospin_cg_beta_gamma = float(
#                 CG(0.5, tau_beta, 0.5, tau_gamma, T, M_T).doit()
#             )
#             # Compute < T M_T | \tau_\alpha \tau_\gamma >
#             isospin_cg_alpha_gamma = float(
#                 CG(0.5, tau_alpha, 0.5, tau_gamma, T, M_T).doit()
#             )
    
#             if S == 0:
            
#                 M_S = 0
#                 M_Lp = M_J - M_S
#                 M_L = M_Lp
                
#                 # Calculate < L' M_L' S M_S | J M_J >
#                 Lp_S_J_cg = float(
#                     CG(Lp, M_Lp, S, M_S, J, M_J).doit()
#                 )
#                 # Calculate < J M_J | L M_L S M_S >
#                 L_S_J_cg = float(
#                     CG(L, M_L, S, M_S, J, M_J).doit()
#                 )
                
#                 # Compute Y^*_{L' M_L'}(\kphat)
#                 # Compute Y_{L M_L}(\khat)
                
#                 for sigma_alpha in spins:
                    
#                     m_l_alpha = m_j_alpha - sigma_alpha

#                     # Compute < l_\alpha m_l_\alpha 1/2 \sigma_\alpha | j_\alpha m_j_\alpha >
#                     alpha_cg = float(
#                         CG(l_alpha, m_l_alpha, 1/2, sigma_alpha, j_alpha, m_j_alpha).doit()
#                     )
                    
#                     # Compute Y_{l_\alpha m_l_\alpha}(\k3hat)
                    
#                     for sigma_beta in spins:
#                         for sigma_betap in spins:
#                             sigma_gamma = M_S - sigma_alpha
#                             sigma_gammap = M_S - sigma_betap
                            
#                             # Compute < \sigma_\beta' \sigma_\gamma' | S M_S >
#                             spin_cg_betap_gammap = float(
#                                 CG(1/2, sigma_betap, 1/2, sigma_gammap, S, M_S).doit()
#                             )
#                             # Compute < S M_S | \sigma_\alpha \sigma_\gamma >
#                             spin_cg_alpha_gamma = float(
#                                 CG(1/2, sigma_alpha, 1/2, sigma_gamma, S, M_S).doit()
#                             )
                        
#                             for gamma in gammas:
                            
#                                 n_gamma, l_gamma, j_gamma = get_sp_quantum_numbers(gamma)
                                
#                                 # Compute \phi^*_\gamma(k2)
#                                 # Compute \phi_\gamma(k4)
                                
#                                 m_j_gammas = np.arange(-j_gamma, j_gamma+1, 1)
#                                 for m_j_gamma in m_j_gammas:
                                
#                                     m_l_gamma = m_j_gamma - sigma_gamma
#                                     m_l_gammap = m_j_gamma - sigma_gammap

#                                     # Compute < j_\gamma m_j_\gamma | l_\gamma m_l_\gamma' 1/2 \sigma_\gamma' >
#                                     gammap_cg = float(
#                                         CG(l_gamma, m_l_gammap, 1/2, sigma_gammap, j_gamma, m_j_gamma).doit()
#                                     )
#                                     # Compute < l_\gamma m_l_\gamma 1/2 \sigma_\gamma | j_\gamma m_j_\gamma >
#                                     gamma_cg = float(
#                                         CG(l_gamma, m_l_gamma, 1/2, sigma_gamma, j_gamma, m_j_gamma).doit()
#                                     )
                                    
#                                     # Compute Y^*_{l_\gamma m_l_\gamma'}(\k2hat)
#                                     # Compute Y_{l_\gamma m_l_\gamma}(\k4hat)
                                
#                                     for beta in betas:
                                    
#                                         n_beta, l_beta, j_beta = get_sp_quantum_numbers(beta)
                                        
#                                         # Compute \phi^*_\beta(k1)
#                                         # Compute \phi_\beta(q)
                                    
#                                         m_j_betas = np.arange(-j_beta, j_beta+1, 1)
#                                         for m_j_beta in m_j_betas:
                                        
#                                             m_l_beta = m_j_beta - sigma_beta
#                                             m_l_betap = m_j_beta - sigma_betap

#                                             # Compute < j_\beta m_j_\beta | l_\beta m_l_\beta' 1/2 \sigma_\beta' >
#                                             betap_cg = float(
#                                                 CG(l_beta, m_l_betap, 1/2, sigma_betap, j_beta, m_j_beta).doit()
#                                             )
#                                             # Compute < l_\beta m_l_\beta 1/2 \sigma_\beta | j_\beta m_j_\beta >
#                                             beta_cg = float(
#                                                 CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit()
#                                             )
                                            
#                                             # Compute Y^*_{l_\beta m_l_\beta'}(\k1hat)
#                                             # Compute Y_{l_\beta m_l_\beta}(\qhat)
                                            
#                                             integrand = (
#                                                 betap_cg * gammap_cg * alpha_cg
#                                                 * beta_cg * gamma_cg
#                                                 * spin_cg_betap_gammap
#                                                 * spin_cg_alpha_gamma
#                                                 * isospin_cg_beta_gamma
#                                                 * isospin_cg_alpha_gamma
#                                                 * Lp_S_J_cg * L_S_J_cg * factor
#                                             )
                                            
#                                             print(integrand)
                                            
# #                                             if integrand == 0:
# #                                                 print_quantum_numbers(
# #                                                     channel, J, M_J, Lp, M_Lp, L,
# #                                                     M_L, S, M_S, T, M_T, tau_alpha,
# #                                                     tau_beta, tau_gamma,
# #                                                     sigma_alpha, sigma_beta,
# #                                                     sigma_gamma, sigma_betap,
# #                                                     sigma_gammap, n_gamma, j_gamma,
# #                                                     m_j_gamma, l_gamma, m_l_gamma,
# #                                                     m_l_gammap, n_beta, j_beta,
# #                                                     m_j_beta, l_beta, m_l_beta,
# #                                                     m_l_betap
# #                                                 )

#                                             if integrand == 0:
#                                                 if betap_cg == 0:
#                                                     print(
#                                                         f"j_beta={j_beta},m_j_beta={m_j_beta},"
#                                                         f"l_beta={l_beta},m_l_beta'={m_l_betap},"
#                                                         f"sigma_beta'={sigma_betap}"
#                                                     )
#                                                 if gammap_cg == 0:
#                                                     print('gamma_cg')
#                                                 if alpha_cg == 0:
#                                                     print('alpha_cg')
#                                                 if gamma_cg == 0:
#                                                     print(gamma_cg)
#                                                 if spin_cg_betap_gammap == 0:
#                                                     print('spin_cg_betap_gammap')
#                                                 if spin_cg_alpha_gamma == 0:
#                                                     print('spin_cg_alpha_gamma')
#                                                 if isospin_cg_beta_gamma == 0:
#                                                     print('isospin_cg_beta_gamma')
#                                                 if isospin_cg_alpha_gamma == 0:
#                                                     print('isospin_cg_alpha_gamma')
#                                                 if Lp_S_J_cg == 0:
#                                                     print('Lp_S_J_cg')
#                                                 if L_S_J_cg == 0:
#                                                     print('L_S_J_cg')
#                                                 if factor == 0:
#                                                     print('factor')

#                                             i += 1

#             else:
                
#                 for M_S in total_spins:
                    
#                     M_Lp = M_J - M_S
#                     M_L = M_Lp
                    
#                     # Calculate < L' M_L' S M_S | J M_J >
#                     Lp_S_J_cg = float(
#                         CG(Lp, M_Lp, S, M_S, J, M_J).doit()
#                     )
#                     # Calculate < J M_J | L M_L S M_S >
#                     L_S_J_cg = float(
#                         CG(L, M_L, S, M_S, J, M_J).doit()
#                     )
                    
#                     # Compute Y^*_{L' M_L'}(\kphat)
#                     # Compute Y_{L M_L}(\khat)
                    
#                     for sigma_alpha in spins:
                        
#                         m_l_alpha = m_j_alpha - sigma_alpha
                        
#                         # Compute < l_\alpha m_l_\alpha 1/2 \sigma_\alpha | j_\alpha m_j_\alpha >
#                         alpha_cg = float(
#                             CG(l_alpha, m_l_alpha, 1/2, sigma_alpha, j_alpha, m_j_alpha).doit()
#                         )
                    
#                         # Compute Y_{l_\alpha m_l_\alpha}(\k3hat)
                        
#                         for sigma_beta in spins:
#                             for sigma_betap in spins:
#                                 sigma_gamma = M_S - sigma_alpha
#                                 sigma_gammap = M_S - sigma_betap
                                
#                                 # Compute < \sigma_\beta' \sigma_\gamma' | S M_S >
#                                 spin_cg_betap_gammap = float(
#                                     CG(1/2, sigma_betap, 1/2, sigma_gammap, S, M_S).doit()
#                                 )
#                                 # Compute < S M_S | \sigma_\alpha \sigma_\gamma >
#                                 spin_cg_alpha_gamma = float(
#                                     CG(1/2, sigma_alpha, 1/2, sigma_gamma, S, M_S).doit()
#                                 )
                            
#                                 for gamma in gammas:
                            
#                                     n_gamma, l_gamma, j_gamma = get_sp_quantum_numbers(gamma)
                                    
#                                     # Compute \phi^*_\gamma(k2)
#                                     # Compute \phi_\gamma(k4)
                            
#                                     m_j_gammas = np.arange(-j_gamma, j_gamma+1, 1)
#                                     for m_j_gamma in m_j_gammas:
                                
#                                         m_l_gamma = m_j_gamma - sigma_gamma
#                                         m_l_gammap = m_j_gamma - sigma_gammap
                                        
#                                         # Compute < j_\gamma m_j_\gamma | l_\gamma m_l_\gamma' 1/2 \sigma_\gamma' >
#                                         gammap_cg = float(
#                                             CG(l_gamma, m_l_gammap, 1/2, sigma_gammap, j_gamma, m_j_gamma).doit()
#                                         )
#                                         # Compute < l_\gamma m_l_\gamma 1/2 \sigma_\gamma | j_\gamma m_j_\gamma >
#                                         gamma_cg = float(
#                                             CG(l_gamma, m_l_gamma, 1/2, sigma_gamma, j_gamma, m_j_gamma).doit()
#                                         )
                                        
#                                         # Compute Y^*_{l_\gamma m_l_\gamma'}(\k2hat)
#                                         # Compute Y_{l_\gamma m_l_\gamma}(\k4hat)
                                
#                                         for beta in betas:
                                    
#                                             n_beta, l_beta, j_beta = get_sp_quantum_numbers(beta)
                                        
#                                             # Compute \phi^*_\beta(k1)
#                                             # Compute \phi_\beta(q)
                                            
#                                             m_j_betas = np.arange(-j_beta, j_beta+1, 1)
#                                             for m_j_beta in m_j_betas:
                                        
#                                                 m_l_beta = m_j_beta - sigma_beta
#                                                 m_l_betap = m_j_beta - sigma_betap
                                                
#                                                 # Compute < j_\beta m_j_\beta | l_\beta m_l_\beta' 1/2 \sigma_\beta' >
#                                                 betap_cg = float(
#                                                     CG(l_beta, m_l_betap, 1/2, sigma_betap, j_beta, m_j_beta).doit()
#                                                 )
#                                                 # Compute < l_\beta m_l_\beta 1/2 \sigma_\beta | j_\beta m_j_\beta >
#                                                 beta_cg = float(
#                                                     CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit()
#                                                 )
                                                
#                                                 # Compute Y^*_{l_\beta m_l_\beta'}(\k1hat)
#                                                 # Compute Y_{l_\beta m_l_\beta}(\qhat)
                                                
#                                                 integrand = (
#                                                     betap_cg * gammap_cg
#                                                     * alpha_cg * beta_cg
#                                                     * gamma_cg
#                                                     * spin_cg_betap_gammap
#                                                     * spin_cg_alpha_gamma
#                                                     * isospin_cg_beta_gamma
#                                                     * isospin_cg_alpha_gamma
#                                                     * Lp_S_J_cg * L_S_J_cg
#                                                     * factor
#                                                 )
                                                
#                                                 print(integrand)
                                                
#                                                 if integrand == 0:
#                                                     if betap_cg == 0:
#                                                         print(
#                                                             f"j_beta={j_beta},m_j_beta={m_j_beta},"
#                                                             f"l_beta={l_beta},m_l_beta'={m_l_betap},"
#                                                             f"sigma_beta'={sigma_betap}"
#                                                         )
#                                                     if gammap_cg == 0:
#                                                         print('gamma_cg')
#                                                     if alpha_cg == 0:
#                                                         print('alpha_cg')
#                                                     if gamma_cg == 0:
#                                                         print(gamma_cg)
#                                                     if spin_cg_betap_gammap == 0:
#                                                         print('spin_cg_betap_gammap')
#                                                     if spin_cg_alpha_gamma == 0:
#                                                         print('spin_cg_alpha_gamma')
#                                                     if isospin_cg_beta_gamma == 0:
#                                                         print('isospin_cg_beta_gamma')
#                                                     if isospin_cg_alpha_gamma == 0:
#                                                         print('isospin_cg_alpha_gamma')
#                                                     if Lp_S_J_cg == 0:
#                                                         print('Lp_S_J_cg')
#                                                     if L_S_J_cg == 0:
#                                                         print('L_S_J_cg')
#                                                     if factor == 0:
#                                                         print('factor')

# #                                                 if integrand == 0:
# #                                                     print_quantum_numbers(
# #                                                         channel, J, M_J, Lp, M_Lp, L,
# #                                                         M_L, S, M_S, T, M_T, tau_alpha,
# #                                                         tau_beta, tau_gamma,
# #                                                         sigma_alpha, sigma_beta,
# #                                                         sigma_gamma, sigma_betap,
# #                                                         sigma_gammap, n_gamma, j_gamma,
# #                                                         m_j_gamma, l_gamma, m_l_gamma,
# #                                                         m_l_gammap, n_beta, j_beta,
# #                                                         m_j_beta, l_beta, m_l_beta,
# #                                                         m_l_betap
# #                                                     )

#                                                 i += 1
            
#         else:
            
#             for tau_gamma in isospins:
            
#                 M_T = tau_alpha + tau_gamma
                
#                 # Compute < \tau_\beta \tau_\gamma | T M_T >
#                 isospin_cg_beta_gamma = float(
#                     CG(0.5, tau_beta, 0.5, tau_gamma, T, M_T).doit()
#                 )
#                 # Compute < T M_T | \tau_\alpha \tau_\gamma >
#                 isospin_cg_alpha_gamma = float(
#                     CG(0.5, tau_alpha, 0.5, tau_gamma, T, M_T).doit()
#                 )
            
#                 if S == 0:
                    
#                     M_S = 0
#                     M_Lp = M_J - M_S
#                     M_L = M_Lp
                    
#                     # Calculate < L' M_L' S M_S | J M_J >
#                     Lp_S_J_cg = float(
#                         CG(Lp, M_Lp, S, M_S, J, M_J).doit()
#                     )
#                     # Calculate < J M_J | L M_L S M_S >
#                     L_S_J_cg = float(
#                         CG(L, M_L, S, M_S, J, M_J).doit()
#                     )
                    
#                     # Compute Y^*_{L' M_L'}(\kphat)
#                     # Compute Y_{L M_L}(\khat)
                    
#                     for sigma_alpha in spins:
                        
#                         m_l_alpha = m_j_alpha - sigma_alpha
                        
#                         # Compute < l_\alpha m_l_\alpha 1/2 \sigma_\alpha | j_\alpha m_j_\alpha >
#                         alpha_cg = float(
#                             CG(l_alpha, m_l_alpha, 1/2, sigma_alpha, j_alpha, m_j_alpha).doit()
#                         )
                    
#                         # Compute Y_{l_\alpha m_l_\alpha}(\k3hat)
                        
#                         for sigma_beta in spins:
#                             for sigma_betap in spins:
#                                 sigma_gamma = M_S - sigma_alpha
#                                 sigma_gammap = M_S - sigma_betap
                                
#                                 # Compute < \sigma_\beta' \sigma_\gamma' | S M_S >
#                                 spin_cg_betap_gammap = float(
#                                     CG(1/2, sigma_betap, 1/2, sigma_gammap, S, M_S).doit()
#                                 )
#                                 # Compute < S M_S | \sigma_\alpha \sigma_\gamma >
#                                 spin_cg_alpha_gamma = float(
#                                     CG(1/2, sigma_alpha, 1/2, sigma_gamma, S, M_S).doit()
#                                 )
                            
#                                 for gamma in gammas:
                            
#                                     n_gamma, l_gamma, j_gamma = get_sp_quantum_numbers(gamma)
                                    
#                                     # Compute \phi^*_\gamma(k2)
#                                     # Compute \phi_\gamma(k4)
                            
#                                     m_j_gammas = np.arange(-j_gamma, j_gamma+1, 1)
#                                     for m_j_gamma in m_j_gammas:
                                
#                                         m_l_gamma = m_j_gamma - sigma_gamma
#                                         m_l_gammap = m_j_gamma - sigma_gammap
                                        
#                                         # Compute < j_\gamma m_j_\gamma | l_\gamma m_l_\gamma' 1/2 \sigma_\gamma' >
#                                         gammap_cg = float(
#                                             CG(l_gamma, m_l_gammap, 1/2, sigma_gammap, j_gamma, m_j_gamma).doit()
#                                         )
#                                         # Compute < l_\gamma m_l_\gamma 1/2 \sigma_\gamma | j_\gamma m_j_\gamma >
#                                         gamma_cg = float(
#                                             CG(l_gamma, m_l_gamma, 1/2, sigma_gamma, j_gamma, m_j_gamma).doit()
#                                         )
                                        
#                                         # Compute Y^*_{l_\gamma m_l_\gamma'}(\k2hat)
#                                         # Compute Y_{l_\gamma m_l_\gamma}(\k4hat)
                                
#                                         for beta in betas:
                                    
#                                             n_beta, l_beta, j_beta = get_sp_quantum_numbers(beta)
                                        
#                                             # Compute \phi^*_\beta(k1)
#                                             # Compute \phi_\beta(q)
                                            
#                                             m_j_betas = np.arange(-j_beta, j_beta+1, 1)
#                                             for m_j_beta in m_j_betas:
                                        
#                                                 m_l_beta = m_j_beta - sigma_beta
#                                                 m_l_betap = m_j_beta - sigma_betap
                                                
#                                                 # Compute < j_\beta m_j_\beta | l_\beta m_l_\beta' 1/2 \sigma_\beta' >
#                                                 betap_cg = float(
#                                                     CG(l_beta, m_l_betap, 1/2, sigma_betap, j_beta, m_j_beta).doit()
#                                                 )
#                                                 # Compute < l_\beta m_l_\beta 1/2 \sigma_\beta | j_\beta m_j_\beta >
#                                                 beta_cg = float(
#                                                     CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit()
#                                                 )
                                                
#                                                 # Compute Y^*_{l_\beta m_l_\beta'}(\k1hat)
#                                                 # Compute Y_{l_\beta m_l_\beta}(\qhat)
                                                
#                                                 integrand = (
#                                                     betap_cg * gammap_cg
#                                                     * alpha_cg * beta_cg
#                                                     * gamma_cg
#                                                     * spin_cg_betap_gammap
#                                                     * spin_cg_alpha_gamma
#                                                     * isospin_cg_beta_gamma
#                                                     * isospin_cg_alpha_gamma
#                                                     * Lp_S_J_cg * L_S_J_cg
#                                                     * factor
#                                                 )
                                                
#                                                 print(integrand)

# #                                                 if integrand == 0:
# #                                                     print_quantum_numbers(
# #                                                         channel, J, M_J, Lp, M_Lp, L,
# #                                                         M_L, S, M_S, T, M_T, tau_alpha,
# #                                                         tau_beta, tau_gamma,
# #                                                         sigma_alpha, sigma_beta,
# #                                                         sigma_gamma, sigma_betap,
# #                                                         sigma_gammap, n_gamma, j_gamma,
# #                                                         m_j_gamma, l_gamma, m_l_gamma,
# #                                                         m_l_gammap, n_beta, j_beta,
# #                                                         m_j_beta, l_beta, m_l_beta,
# #                                                         m_l_betap
# #                                                     )

#                                                 if integrand == 0:
#                                                     if betap_cg == 0:
#                                                         print(
#                                                             f"j_beta={j_beta},m_j_beta={m_j_beta},"
#                                                             f"l_beta={l_beta},m_l_beta'={m_l_betap},"
#                                                             f"sigma_beta'={sigma_betap}"
#                                                         )
#                                                     if gammap_cg == 0:
#                                                         print('gamma_cg')
#                                                     if alpha_cg == 0:
#                                                         print('alpha_cg')
#                                                     if gamma_cg == 0:
#                                                         print(gamma_cg)
#                                                     if spin_cg_betap_gammap == 0:
#                                                         print('spin_cg_betap_gammap')
#                                                     if spin_cg_alpha_gamma == 0:
#                                                         print('spin_cg_alpha_gamma')
#                                                     if isospin_cg_beta_gamma == 0:
#                                                         print('isospin_cg_beta_gamma')
#                                                     if isospin_cg_alpha_gamma == 0:
#                                                         print('isospin_cg_alpha_gamma')
#                                                     if Lp_S_J_cg == 0:
#                                                         print('Lp_S_J_cg')
#                                                     if L_S_J_cg == 0:
#                                                         print('L_S_J_cg')
#                                                     if factor == 0:
#                                                         print('factor')

#                                                 i += 1

#                 else:
                
#                     for M_S in total_spins:
                        
#                         M_Lp = M_J - M_S
#                         M_L = M_Lp
                        
#                         # Calculate < L' M_L' S M_S | J M_J >
#                         Lp_S_J_cg = float(
#                             CG(Lp, M_Lp, S, M_S, J, M_J).doit()
#                         )
#                         # Calculate < J M_J | L M_L S M_S >
#                         L_S_J_cg = float(
#                             CG(L, M_L, S, M_S, J, M_J).doit()
#                         )
                        
#                         # Compute Y^*_{L' M_L'}(\kphat)
#                         # Compute Y_{L M_L}(\khat)
                        
#                         for sigma_alpha in spins:
                            
#                             m_l_alpha = m_j_alpha - sigma_alpha
                            
#                             # Compute < l_\alpha m_l_\alpha 1/2 \sigma_\alpha | j_\alpha m_j_\alpha >
#                             alpha_cg = float(
#                                 CG(l_alpha, m_l_alpha, 1/2, sigma_alpha, j_alpha, m_j_alpha).doit()
#                             )
                    
#                             # Compute Y_{l_\alpha m_l_\alpha}(\k3hat)
                            
#                             for sigma_beta in spins:
#                                 for sigma_betap in spins:
#                                     sigma_gamma = M_S - sigma_alpha
#                                     sigma_gammap = M_S - sigma_betap
                                    
#                                     # Compute < \sigma_\beta' \sigma_\gamma' | S M_S >
#                                     spin_cg_betap_gammap = float(
#                                         CG(1/2, sigma_betap, 1/2, sigma_gammap, S, M_S).doit()
#                                     )
#                                     # Compute < S M_S | \sigma_\alpha \sigma_\gamma >
#                                     spin_cg_alpha_gamma = float(
#                                         CG(1/2, sigma_alpha, 1/2, sigma_gamma, S, M_S).doit()
#                                     )
                                
#                                     for gamma in gammas:
                            
#                                         n_gamma, l_gamma, j_gamma = get_sp_quantum_numbers(gamma)
                                    
#                                         # Compute \phi^*_\gamma(k2)
#                                         # Compute \phi_\gamma(k4)
                                    
#                                         m_j_gammas = np.arange(-j_gamma, j_gamma+1, 1)
#                                         for m_j_gamma in m_j_gammas:
                                
#                                             m_l_gamma = m_j_gamma - sigma_gamma
#                                             m_l_gammap = m_j_gamma - sigma_gammap
                                        
#                                             # Compute < j_\gamma m_j_\gamma | l_\gamma m_l_\gamma' 1/2 \sigma_\gamma' >
#                                             gammap_cg = float(
#                                                 CG(l_gamma, m_l_gammap, 1/2, sigma_gammap, j_gamma, m_j_gamma).doit()
#                                             )
#                                             # Compute < l_\gamma m_l_\gamma 1/2 \sigma_\gamma | j_\gamma m_j_\gamma >
#                                             gamma_cg = float(
#                                                 CG(l_gamma, m_l_gamma, 1/2, sigma_gamma, j_gamma, m_j_gamma).doit()
#                                             )
                                        
#                                             # Compute Y^*_{l_\gamma m_l_\gamma'}(\k2hat)
#                                             # Compute Y_{l_\gamma m_l_\gamma}(\k4hat)
                                
#                                             for beta in betas:
                                    
#                                                 n_beta, l_beta, j_beta = get_sp_quantum_numbers(beta)
                                        
#                                                 # Compute \phi^*_\beta(k1)
#                                                 # Compute \phi_\beta(q)
                                        
#                                                 m_j_betas = np.arange(-j_beta, j_beta+1, 1)
#                                                 for m_j_beta in m_j_betas:
                                            
#                                                     m_l_beta = m_j_beta - sigma_beta
#                                                     m_l_betap = m_j_beta - sigma_betap
                                                    
#                                                     # Compute < j_\beta m_j_\beta | l_\beta m_l_\beta' 1/2 \sigma_\beta' >
#                                                     betap_cg = float(
#                                                         CG(l_beta, m_l_betap, 1/2, sigma_betap, j_beta, m_j_beta).doit()
#                                                     )
#                                                     # Compute < l_\beta m_l_\beta 1/2 \sigma_\beta | j_\beta m_j_\beta >
#                                                     beta_cg = float(
#                                                         CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit()
#                                                     )
                                                
#                                                     # Compute Y^*_{l_\beta m_l_\beta'}(\k1hat)
#                                                     # Compute Y_{l_\beta m_l_\beta}(\qhat)
                                                
#                                                     integrand = (
#                                                         betap_cg * gammap_cg
#                                                         * alpha_cg * beta_cg
#                                                         * gamma_cg
#                                                         * spin_cg_betap_gammap
#                                                         * spin_cg_alpha_gamma
#                                                         * isospin_cg_beta_gamma
#                                                         * isospin_cg_alpha_gamma
#                                                         * Lp_S_J_cg * L_S_J_cg
#                                                         * factor
#                                                     )
                                                    
#                                                     print(integrand)

# #                                                     if integrand == 0:
# #                                                         print_quantum_numbers(
# #                                                             channel, J, M_J, Lp, M_Lp, L,
# #                                                             M_L, S, M_S, T, M_T, tau_alpha,
# #                                                             tau_beta, tau_gamma,
# #                                                             sigma_alpha, sigma_beta,
# #                                                             sigma_gamma, sigma_betap,
# #                                                             sigma_gammap, n_gamma, j_gamma,
# #                                                             m_j_gamma, l_gamma, m_l_gamma,
# #                                                             m_l_gammap, n_beta, j_beta,
# #                                                             m_j_beta, l_beta, m_l_beta,
# #                                                             m_l_betap
# #                                                         )

#                                                     if integrand == 0:
#                                                         if betap_cg == 0:
#                                                             print(
#                                                                 f"j_beta={j_beta},m_j_beta={m_j_beta},"
#                                                                 f"l_beta={l_beta},m_l_beta'={m_l_betap},"
#                                                                 f"sigma_beta'={sigma_betap}"
#                                                             )
#                                                         if gammap_cg == 0:
#                                                             print('gamma_cg')
#                                                         if alpha_cg == 0:
#                                                             print('alpha_cg')
#                                                         if gamma_cg == 0:
#                                                             print(gamma_cg)
#                                                         if spin_cg_betap_gammap == 0:
#                                                             print('spin_cg_betap_gammap')
#                                                         if spin_cg_alpha_gamma == 0:
#                                                             print('spin_cg_alpha_gamma')
#                                                         if isospin_cg_beta_gamma == 0:
#                                                             print('isospin_cg_beta_gamma')
#                                                         if isospin_cg_alpha_gamma == 0:
#                                                             print('isospin_cg_alpha_gamma')
#                                                         if Lp_S_J_cg == 0:
#                                                             print('Lp_S_J_cg')
#                                                         if L_S_J_cg == 0:
#                                                             print('L_S_J_cg')
#                                                         if factor == 0:
#                                                             print('factor')

#                                                     i += 1
                                                
# print(f"i={i}")

In [16]:
# Example of how to use NumPy intersect1d to constrain quantum numbers

# for l_beta in np.arange(0, 3):
#     for m_l_beta in np.arange(-l_beta, l_beta+1, 1):
#         for j_beta in np.arange(abs(l_beta-1/2), l_beta+3/2, 1):
#             for m_j_beta in np.intersect1d(
#                 np.arange(-j_beta, j_beta+1, 1),
#                 np.array((m_l_beta-1/2, m_l_beta+1/2))
#             ):
#                 sigma_beta = m_j_beta-m_l_beta
#                 print(f"l_beta={l_beta}, m_l_beta={m_l_beta}, j_beta={j_beta}, "
#                       f"m_j_beta={m_j_beta}, sigma_beta={sigma_beta}, "
#                       f"CG = {float(CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit())}")

In [17]:
# Todo:
# * First check with momentum and angle dependence exactly the same as before:
#   k, k', and K with angle-averaging integrals. Use multi-dimensioned arrays.
# * Second, calculate using vegas up to \beta >= 1s_{1/2} and channels 1S0 and
#   3S1-3S1, and compare to previous calculation.

In [18]:
alpha = '1s0p5'
tau_alpha = 1/2

# channels = (
#     '1S0', '3S1-3S1', '3S1-3D1', '3D1-3S1', '3D1-3D1', '3P0', '3P1', '1P1'
# )
channels = ('1S0', '3S1-3S1')

spins = np.array([1/2, -1/2])
isospins = np.array([1/2, -1/2])

total_spins = np.array([1, 0, -1])

# betas = ['1s0p5', '1p1p5', '1p0p5', '2s0p5', '2p1p5', '2p0p5']
betas = ['1s0p5']
gammas = ['1s0p5']

n_alpha, l_alpha, j_alpha = get_sp_quantum_numbers(alpha)
m_j_alpha = j_alpha

In [19]:
# Second attempt

# Count the number of non-zero contributions
i = 0

tau_beta = tau_alpha

# Compute k4_vector, k4, \theta_4, \phi_4
# Compute kp_vector, kp, \theta', \phi'
# Compute k_vector, k, \theta, \phi

# Compute \phi_\alpha(k3)

## ----- TESTING ----- ##
# Initialize momentum and angle meshes
k_array, k_weights = momentum_mesh(10.0, 2.0, 120)  # Relative momenta
K_array, K_weights = gaussian_quadrature_mesh(2.0, 40)  # CoM momenta
x_array, x_weights = leggauss(7)  # Angles between K and k' (or K and k)
    
# Get meshgrids for (k', k, K, x)
kp_grid, k_grid, K_grid, x_grid = np.meshgrid(k_array, k_array, K_array,
                                              x_array, indexing='ij')
_, _, _, dx_grid = np.meshgrid(k_weights, k_weights, K_weights, x_weights,
                               indexing='ij')

phi_p_func = interpolate_phi_kspace('proton')
phi_n_func = interpolate_phi_kspace('neutron')
## ----- TESTING ----- ##

# Loop over channels
for channel in channels:
    
    # Determine L', L, S, J, T
    S, Lp, L, J = get_channel_quantum_numbers(channel)
    T = get_total_isospin(S, Lp)
    
    total_angular_momentum_projections = np.arange(-J, J+1, 1)
    total_isospin_projections = np.intersect1d(np.arange(-T, T+1, 1),
                                               tau_alpha - isospins)
    
    # Compute 1-(-1)^(L'+S+T) and 1-(-1)^(L+S+T) factors
    factor = (1-(-1)**(Lp+S+T)) * (1-(-1)**(L+S+T))
    
    # Compute \delta U^\dagger(k', k)
    
    # Loop over M_T
    for M_T in total_isospin_projections:
    
        # Determine \tau_\gamma (\tau_\alpha already known)
        tau_gamma = M_T - tau_beta

        # Compute < \tau_\beta \tau_\gamma | T M_T >
        isospin_cg_beta_gamma = float(
            CG(0.5, tau_beta, 0.5, tau_gamma, T, M_T).doit()
        )
        
        # Compute < T M_T | \tau_\alpha \tau_\gamma >
        isospin_cg_alpha_gamma = float(
            CG(0.5, tau_alpha, 0.5, tau_gamma, T, M_T).doit()
        )
        
        for beta in betas:

            # Determine n_\beta, l_\beta, j_\beta
            n_beta, l_beta, j_beta = get_sp_quantum_numbers(beta)
            
            # Compute \phi_\beta(k1), \phi_\beta(q)
            
            # Loop over \gamma
            for gamma in gammas:
            
                # Determine n_\gamma, l_\gamma, j_\gamma
                n_gamma, l_gamma, j_gamma = get_sp_quantum_numbers(gamma)
                
                # Compute \phi_\gamma(k2), \phi_\gamma(k4)
                
                # Loop over m_l_\alpha
                m_l_alphas = np.intersect1d(np.arange(-l_alpha, l_alpha+1, 1),
                                            m_j_alpha - spins)
                for m_l_alpha in m_l_alphas:

                    # Compute Y_{m_l_\alpha}(k3)
                    
                    # Determine \sigma_\alpha
                    sigma_alpha = m_j_alpha - m_l_alpha
                    
                    # Compute < l_\alpha m_l_\alpha 1/2 \sigma_\alpha | j_\alpha m_j_\alpha >
                    alpha_cg = float(
                        CG(l_alpha, m_l_alpha, 1/2, sigma_alpha, j_alpha, m_j_alpha).doit()
                    )
                    
                    # Loop over m_j_\beta
                    for m_j_beta in np.arange(-j_beta, j_beta+1, 1):
                
                        # Loop over m_l_\beta'
                        m_l_betas = np.intersect1d(
                            np.arange(-l_beta, l_beta+1, 1), m_j_beta - spins
                        )
                        for m_l_betap in m_l_betas:
                    
                            # Compute Y_{m_l_\beta'}(k1)
                        
                            # Determine \sigma_\beta'
                            sigma_betap = m_j_beta - m_l_betap
                            
                            # Compute < j_\beta m_j_\beta | l_\beta m_l_\beta' 1/2 \sigma_\beta' >
                            betap_cg = float(
                                CG(l_beta, m_l_betap, 1/2, sigma_betap, j_beta, m_j_beta).doit()
                            )
                
                            # Loop over m_l_\beta
                            for m_l_beta in m_l_betas:
                        
                                # Compute Y_{m_l_\beta}(q)
                            
                                # Determine \sigma_\beta
                                sigma_beta = m_j_beta - m_l_beta
                                
                                # Compute < l_\beta m_l_\beta 1/2 \sigma_\beta | j_\beta m_j_\beta >
                                beta_cg = float(
                                    CG(l_beta, m_l_beta, 1/2, sigma_beta, j_beta, m_j_beta).doit()
                                )
                            
                                # Loop over m_j_\gamma
                                for m_j_gamma in np.arange(-j_gamma, j_gamma+1, 1):
                            
                                    # Loop over m_l_\gamma'
                                    m_l_gammas = np.intersect1d(
                                        np.arange(-l_gamma, l_gamma+1, 1), m_j_gamma - spins
                                    )
                                    for m_l_gammap in m_l_gammas:
                                
                                        # Compute Y_{m_l_\gamma'}(k2)
                                    
                                        # Determine \sigma_\gamma'
                                        sigma_gammap = m_j_gamma - m_l_gammap
                                        
                                        # Compute < j_\gamma m_j_\gamma | l_\gamma m_l_\gamma' 1/2 \sigma_\gamma' >
                                        gammap_cg = float(
                                            CG(l_gamma, m_l_gammap, 1/2, sigma_gammap, j_gamma, m_j_gamma).doit()
                                        )
                                        
                                        # Loop over m_l_\gamma
                                        for m_l_gamma in m_l_gammas:

                                            # Compute Y_{m_l_\gamma}(k4)
                                        
                                            # Determine \sigma_\gamma
                                            sigma_gamma = m_j_gamma - m_l_gamma
                                        
                                            # Compute < l_\gamma m_l_\gamma 1/2 \sigma_\gamma | j_\gamma m_j_\gamma >
                                            gamma_cg = float(
                                                CG(l_gamma, m_l_gamma, 1/2, sigma_gamma, j_gamma, m_j_gamma).doit()
                                            )
                                        
                                            # Loop over M_J and M_L
                                            for M_J in total_angular_momentum_projections:
                                                for M_L in np.arange(-L, L+1, 1):
                                            
                                                    # Determine M_L'
                                                    M_Lp = M_L
                                                
                                                    # Compute Y_{L' M_L'}(k')
                                                    # Compute Y_{L M_L}(k)
                                                
                                                    # Determine M_S
                                                    M_S = M_J - M_L
                                                
                                                    # Compute < \sigma_\beta' \sigma_\gamma' | S M_S >
                                                    spin_cg_betap_gammap = float(
                                                        CG(1/2, sigma_betap, 1/2, sigma_gammap, S, M_S).doit()
                                                    )
                                        
                                                    # Compute < S M_S | \sigma_\alpha \sigma_\gamma >
                                                    spin_cg_alpha_gamma = float(
                                                        CG(1/2, sigma_alpha, 1/2, sigma_gamma, S, M_S).doit()
                                                    )
                                                    # Note, these last two CG's can be zero!
                                                
                                                    # Compute < L' M_L' S M_S | J M_J >
                                                    Lp_S_J_cg = float(
                                                        CG(Lp, M_Lp, S, M_S, J, M_J).doit()
                                                    )
                                
                                                    # Calculate < J M_J | L M_L S M_S >
                                                    L_S_J_cg = float(
                                                        CG(L, M_L, S, M_S, J, M_J).doit()
                                                    )

                                                    # Add to integrand
                                                    integrand = (
                                                        betap_cg * gammap_cg * alpha_cg
                                                        * beta_cg * gamma_cg
                                                        * spin_cg_betap_gammap
                                                        * spin_cg_alpha_gamma
                                                        * isospin_cg_beta_gamma
                                                        * isospin_cg_alpha_gamma
                                                        * Lp_S_J_cg * L_S_J_cg * factor
                                                    )
                                            
                                                    # TESTING
                                                    if integrand != 0:
                                                        print_quantum_numbers(
                                                            channel, J, M_J, Lp, M_Lp, L,
                                                            M_L, S, M_S, T, M_T, tau_alpha,
                                                            tau_beta, tau_gamma,
                                                            sigma_alpha, sigma_beta,
                                                            sigma_gamma, sigma_betap,
                                                            sigma_gammap, n_gamma, j_gamma,
                                                            m_j_gamma, l_gamma, m_l_gamma,
                                                            m_l_gammap, n_beta, j_beta,
                                                            m_j_beta, l_beta, m_l_beta,
                                                            m_l_betap
                                                        )
                                                        i+=1

NameError: name 'momentum_mesh' is not defined

In [ ]:
print(i)